<p><b>Подключение библиотек и функций</b></p>

In [2]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [63]:
def getSE(sigma, n):
    return sigma / np.sqrt(n)

def getSE2(sigma1, n1, sigma2, n2):
    return np.sqrt(sigma1**2 / n1 + sigma2**2 / n2)

def getZ(x, mu, se):
    return (x - mu) / se

def getPrByX(x, mu, se):
    return stats.norm.cdf(x, loc=mu, scale=se)

def getPrByZ(z):
    return stats.norm.cdf(z, loc=0, scale=1)

def getZbyPr2Tail(pr):
    return stats.norm.ppf((1-pr)/2, loc=0, scale=1)

def getZbyPr1Tail(pr):
    return stats.norm.ppf(1-pr, loc=0, scale=1)

def getPValue1Tail_Norm(z):
    return 1 - stats.norm.cdf(z, loc=0, scale=1)

def getPValue2Tail_Norm(z):
    return 2 * (stats.norm.cdf(-abs(z), loc=0, scale=1))

def getPValue2Tail_T(t, df):
    return 2 * stats.t.cdf(-abs(t), df=df)

def getPValue1Tail_T(t, df):
    return 1 - stats.t.cdf(t, df=df)

def get_dof_two(sigma1, n1, sigma2, n2):
    """Calculate a degree of freedom using Welch–Satterthwaite equation."""
    return (sigma1**2/n1 + sigma2**2/n2)**2/((sigma1**2)**2/(n1**2*(n1-1)) + (sigma2**2)**2/(n2**2*(n2-1)))

<p>Создание генеральных совокупностей</p>

In [4]:
mu1 = 70
sigma1 = 30
population1 = stats.norm.rvs(size=10000, loc=mu1, scale=sigma1)

In [5]:
mu2 = 60
sigma2 = 20
population2 = stats.norm.rvs(size=10000, loc=mu2, scale=sigma2)

In [6]:
mu3 = 2
sigma3 = 2
diff = stats.norm.rvs(size=10000, loc=mu3, scale=sigma3)
population3 = list(zip(population1, population1 + diff))

<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [7]:
n = 40
sample_2015 = np.random.choice(population1, n, replace=False)
sample_2015

array([ 50.4253364 ,   6.23665308,  23.06275528,  61.4301444 ,
        91.00665934,  68.40666923,  65.1854169 , 130.40948791,
        57.32094633,  27.34394959,  25.31848483,  72.61078507,
        28.48771724,  68.07844205,  36.22809522, 108.64950876,
        72.65316942,  86.97599776,  60.92592692,  60.2372188 ,
        66.96587548, 118.40948086,  75.88699979,  50.42404383,
        81.88728647,  64.57638051,  67.63648168,  37.80349559,
       101.71606946, -44.50890823,  50.39847818, 107.75170365,
        70.52070489,  45.50563914,  44.44048775,  50.74461774,
        28.95748223,  87.97090598,  53.22357769, 122.73459977])

<p>Статистики выборки</p>

In [8]:
#Среднее значение
x_bar = sample_2015.mean() # Your code here
print(x_bar)

#Выборочная дисперсия
var_2015 = sample_2015.var() # Your code here
print(var_2015)

#Стандартное отклонение (несмещенное)
s_2015 = sample_2015.std() # Your code here
print(s_2015)

62.10096917417411
1094.2233598841754
33.079047142929845


<p><b><i>Доверительный интервал</i></b></p>

In [9]:
#Уровни доверия: 95%, 90%, 80%
prs = np.array([0.95, 0.9, 0.8])

In [10]:
#Стандартная ошибка
se = getSE(s_2015,n) # Your code here
se

5.230256589987185

In [11]:
#Доверительный интервал
from scipy.stats import norm
z = norm.ppf(1 - (1 - prs) / 2)
lowers, uppers = x_bar - z * (s_2015 / np.sqrt(n)) ,x_bar + z * (s_2015 / np.sqrt(n))  # Your code her
for i in range(len(prs)):
    print("Уровень доверия %i%%: (%f, %f)" % (prs[i]*100, lowers[i], uppers[i]))

Уровень доверия 95%: (51.849855, 72.352084)
Уровень доверия 90%: (53.497963, 70.703976)
Уровень доверия 80%: (55.398126, 68.803813)


In [12]:
#Доверительный интервал для t-распределения (при небольших n)
from scipy.stats import t
t = np.array([stats.t.ppf((1 + pr) / 2, n - 1) for pr in prs])
lowers_t, uppers_t = x_bar - t * (s_2015 / np.sqrt(n)) ,x_bar + t * (s_2015 / np.sqrt(n))  # Your code here
print("t-распределение")
for i in range(len(prs)):
    print("Уровень доверия %i%%: (%f, %f)" % (prs[i]*100, lowers_t[i], uppers_t[i]))

t-распределение
Уровень доверия 95%: (51.521777, 72.680162)
Уровень доверия 90%: (53.288640, 70.913298)
Уровень доверия 80%: (55.282605, 68.919333)


<p>или</p>

In [13]:
#z-значения для двустороннего интервала
z = norm.ppf(1 - (1 - prs) / 2)
zs = z / (s_2015 / np.sqrt(n))# Your code here
zs

array([0.37473572, 0.31448813, 0.24502652])

In [15]:
#Предельная ошибка
z = norm.ppf(1 - (1 - prs) / 2)
delta = z * (s_2015 / np.sqrt(n)) # Your code here
delta

array([10.25111455,  8.60300652,  6.70284352])

In [16]:
#Доверительный интервал
lowers = x_bar - z * (s_2015 / np.sqrt(n)) # Your code here
uppers = x_bar + z * (s_2015 / np.sqrt(n)) # Your code here

for i in range(len(prs)):
    print("Уровень доверия %i%%: (%f, %f)" % (prs[i]*100, lowers[i], uppers[i]))

Уровень доверия 95%: (51.849855, 72.352084)
Уровень доверия 90%: (53.497963, 70.703976)
Уровень доверия 80%: (55.398126, 68.803813)


<p>$$H_0: \mu_{2014}=\mu_{2015}$$<p>
<p>$$H_A: \mu_{2014} \neq \mu_{2015}$$<p>

<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [17]:
mu_2014 = 65
n = 40
alpha = 0.05 #уровень значимости
sample_2015 = np.random.choice(population1, n, replace=False)
sample_2015

array([ 86.89471151,  35.75536467,  87.01687118,  65.23113929,
        50.02580198,   2.05959631,  36.89114071,   1.22478153,
       104.57593563,  63.60118673,  70.04518903,  78.42613244,
        86.07395243,  78.58902055,  65.7466731 ,  92.03042017,
        58.25205827,  66.6632773 ,   9.72465031,  94.26120394,
       104.50175341,  39.79731689, 139.35222477,  59.12865461,
        57.11245148,  88.35249342,  91.35040993,  90.0130167 ,
        17.69401333,  75.90024486,  53.04712134,  61.6214508 ,
        81.90187564,  65.50174962,  61.27514707,  51.79674531,
        65.48666361,  79.63404971,  86.17908374,  33.62765162])

In [22]:
#Среднее значение
x_bar_2015 = sample_2015.mean() # Your code here
print(x_bar)

#Стандартное отклонение (несмещенное)
s_2015 = sample_2015.std() # Your code here
print(s_2015)

#Стандартная ошибка
se = getSE(s_2015,n) # Your code here
print(se)

62.10096917417411
28.39073266402354
4.488968982962716


<p><b>Проверка гипотезы</b></p>

In [23]:
#Z-score
z = getZ(x_bar_2015,mu_2014,se) # Your code here
z

0.20251434732380252

In [24]:
#P-Value
pvalue = getPValue1Tail_Norm(z) # Your code here
pvalue

0.4197573216188969

In [40]:
#P-Value для t-распределения
df = n - 1 #степень свободы
s_t_2015 = sample_2015.std(ddof=1)
se_t_2015 = getSE(s_t_2015,n)
t = getZ(x_bar_2015, mu_2014,se_t_2015)
pvalue_t = getPValue2Tail_Norm(t) # Your code here
pvalue_t

0.8415064716470805

<p>или</p>

In [39]:
tvalue, pvalue_t = stats.ttest_1samp(sample_2015,mu_2014) # Your code here
pvalue_t

0.842545531144979

<p><b>Проверка гипотезы на соответствие уровню значимости $\alpha$</b></p>

In [30]:
if alpha > pvalue:
    print("alpha > p-value")
    print("Отказываемся от нулевую гипотезы в пользу альтернативной")
else:
    print("alpha <= p-value")
    print("Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)")

alpha <= p-value
Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)


<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [31]:
alpha = 0.05 #уровень значимости

In [32]:
import random

n_items = 40
sample_items = np.array(random.sample(population3, n_items))
sample_items

array([[ 38.80183238,  38.72246426],
       [ 60.42342482,  62.06969945],
       [ 97.36959005,  98.8980478 ],
       [ 43.07304541,  45.64943551],
       [ 78.48638842,  79.77206487],
       [ 78.3028912 ,  80.62764685],
       [  8.93537618,  11.95813492],
       [ 75.38801848,  78.51815354],
       [ 16.38845332,  19.1063055 ],
       [ 85.29191526,  87.30567076],
       [ 76.0977737 ,  75.48434162],
       [ 82.22129152,  85.32952536],
       [ 22.13217819,  22.40599294],
       [ 47.80226786,  48.30636505],
       [ 78.12907021,  76.97936235],
       [ 27.54963602,  31.76284313],
       [ 98.50705112,  99.95998131],
       [ 63.0434498 ,  64.02850201],
       [ 60.33703563,  62.26219115],
       [ 69.15759709,  73.9702951 ],
       [ 45.31837907,  45.68888813],
       [ 73.33808315,  75.46295415],
       [105.06523037, 104.20543538],
       [ 56.15185224,  58.68103463],
       [103.89863838, 110.59867558],
       [ 48.01867731,  52.00895298],
       [ 96.94726479,  98.90072246],
 

In [33]:
sample_diff = sample_items[:,0] - sample_items[:,1]
sample_diff

array([ 0.07936811, -1.64627464, -1.52845775, -2.5763901 , -1.28567645,
       -2.32475565, -3.02275874, -3.13013505, -2.71785218, -2.0137555 ,
        0.61343208, -3.10823383, -0.27381474, -0.50409719,  1.14970785,
       -4.2132071 , -1.45293018, -0.98505222, -1.92515551, -4.812698  ,
       -0.37050906, -2.124871  ,  0.85979498, -2.52918239, -6.70003719,
       -3.99027567, -1.95345767, -6.87501186, -5.17235268,  0.36524573,
        1.80586824, -3.63267862, -0.18288565, -2.82804403, -5.0770991 ,
       -0.67293903, -1.46080686, -2.64979997,  3.39616439, -5.21948239])

In [47]:
#Среднее значение
x_bar_diff = sample_diff.mean() # Your code here
print("x_bar_diff =", x_bar_diff)

#Стандартное отклонение (несмещенное)
s_diff = sample_diff.std(ddof=1) # Your code here
print("s_diff =", s_diff)

#Стандартная ошибка
se_diff = getSE(s_diff,n) # Your code here
print("se_diff =", se_diff)

x_bar_diff = -2.017277415946659
s_diff = 2.2358266734683223
se_diff = 0.35351523707587285


<p><b>Проверка гипотезы</b></p>

<p>$$H_0: \mu_{diff}=0$$<p>
<p>$$H_A: \mu_{diff} \neq 0$$<p>

In [48]:
#Z-score
z = getZ(x_bar_diff, 0, se_diff) # Your code here
z

-5.706337957686681

In [37]:
#P-Value
pvalue = getPValue2Tail_Norm(z) # Your code here
pvalue

1.1944198466825286e-30

In [51]:
#P-Value для t-распределения
df = n-1 # Your code here
s_t_diff = sample_diff.std(ddof=1)
se_t_diff = getSE(s_t_diff,n)
t = getZ(x_bar_2015, 0,se_t_diff)
pvalue_t = getPValue2Tail_T(t,df) # Your code here
pvalue_t

3.708271247170603e-59

<p>или</p>

In [52]:
tvalue, pvalue_t = stats.ttest_rel(sample_items[:,0],sample_items[:,1]) # Your code here
print("t-value =", tvalue)
print("p-value =", pvalue_t)

t-value = -5.706337957686681
p-value = 1.3246984253014826e-06


In [53]:
pvalue_t = 2 * stats.t.cdf(-abs(tvalue), df)
pvalue_t

1.3246984253014826e-06

<p><b>Проверка гипотезы на соответствие уровню значимости $\alpha$</b></p>

In [54]:
if alpha > pvalue:
    print("alpha > p-value")
    print("Отказываемся от нулевую гипотезы в пользу альтернативной")
else:
    print("alpha <= p-value")
    print("Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)")

alpha > p-value
Отказываемся от нулевую гипотезы в пользу альтернативной


<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [57]:
alpha = 0.05 #уровень значимости

In [58]:
n_london = 40
sample_london = np.random.choice(population1, n_london, replace=False)
sample_london

array([ 48.01493498,  74.51639137,  59.84743666,  25.4179072 ,
        78.44747655,  20.10877125,   4.87721263, 110.61704733,
        17.69401333,  80.90460341,  34.22336712,  62.20565686,
        43.5679352 ,  64.37152439,  71.84047007,  51.90523574,
        32.72200211,  22.30487489,  52.28447614,  55.74739155,
        32.9913622 , 109.83964692,  69.37220739,  82.26371952,
        50.82492582,  22.97336413, 118.18159015, 113.481446  ,
        49.82948874,  38.46730946,  69.32846658,  56.76622463,
        40.44166643,  75.68315349,  76.79778861,  55.35716354,
       101.68390357,  65.80426713,  92.2527116 ,  95.44492585])

In [59]:
n_moscow = 50
sample_moscow = np.random.choice(population2, n_moscow, replace=False)
sample_moscow

array([ 62.53490004,  45.24068939,  34.09497871,  56.10881831,
        59.46141285,  35.96210942,  18.03931509,  46.01049278,
        78.73803847,  67.17060033,  39.1503874 ,  93.22183225,
        33.35486734,  62.53075152,  61.34146157,  96.58716521,
        30.83503039,  84.27935526,  35.64060072,  70.50281215,
        92.47973437,  35.13027164,  53.32217244,  81.42346026,
        43.29653713,  34.49623697,  22.55113586,  80.00189074,
        50.71375343,  62.3643753 ,  33.96821455,  41.14022766,
        82.86879454,  82.84101453,  27.1333984 ,  65.60246413,
        45.2571962 ,  60.99212599,  61.60296207,  42.56563474,
        67.68059858,  28.56678233,  49.2512586 ,  65.40835542,
        54.05782211,  66.05183938, 101.91759065,  52.5935298 ,
        64.44245974,  74.15598118])

In [60]:
#Среднее значение
x_bar_london = sample_london.mean() # Your code here
x_bar_moscow = sample_moscow.mean() # Your code here
print("x_bar_london =", x_bar_london)
print("x_bar_moscow =", x_bar_moscow)

#Стандартное отклонение (несмещенное)
s_london = sample_london.std(ddof=0) # Your code here
s_moscow = sample_moscow.std(ddof=0) # Your code here
print("s_london =", s_london)
print("s_moscow =", s_moscow)

#Стандартная ошибка
se_london_moscow = getSE2(s_london,n_london,s_moscow,n_moscow) # Your code here
print("se_london_moscow =", se_london_moscow)

x_bar_london = 60.73510151454981
x_bar_moscow = 56.69366875918593
s_london = 28.220816743055583
s_moscow = 20.518759046878948
se_london_moscow = 5.322663985036599


<p><b>Проверка гипотезы</b></p>

<p>$$H_0: \mu_{treated}-\mu_{control}=0$$<p>
<p>$$H_A: \mu_{treated}-\mu_{control} \neq 0$$<p>

In [61]:
#Z-score
z = getZ(x_bar_london-x_bar_moscow,0,se_london_moscow) # Your code here
z

0.7592875986020164

In [62]:
#P-Value
pvalue = getPValue2Tail_Norm(z) # Your code here
pvalue

0.44768053437802413

In [64]:
#P-Value для t-распределения
s_t_moscow = sample_moscow.std(ddof=1)
s_t_london = sample_london.std(ddof=1)

se_t_moscow_london = getSE2(s_t_moscow,n_moscow,s_t_london,n_london)
t = getZ(x_bar_london-x_bar_moscow, 0, se_t_moscow_london)
df = get_dof_two(s_t_moscow,n_moscow,s_t_london,n_london) # Your code here
if n_london > n_moscow:
    df = n_moscow - 1

pvalue_t = getPValue2Tail_T(t,df) # Your code here
pvalue_t

0.45562164041991793

<p>или</p>

In [65]:
tvalue, pvalue_t = stats.ttest_ind(sample_london,sample_moscow,equal_var=False) # Your code here
pvalue_t

0.45562164041991793

<p>или</p>

In [66]:
tvalue, pvalue_t = stats.ttest_ind_from_stats(x_bar_london, s_t_london, n_london, x_bar_moscow, s_t_moscow, n_moscow, equal_var=False) # Your code here
pvalue_t

0.45562164041991793

<p><b>Проверка гипотезы на соответствие уровню значимости $\alpha$</b></p>

In [67]:
if alpha > pvalue:
    print("alpha > p-value")
    print("Отказываемся от нулевую гипотезы в пользу альтернативной")
else:
    print("alpha <= p-value")
    print("Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)")

alpha <= p-value
Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)
